In [2]:
import pandas as pd
import random


# Load the first CSV file
df1 = pd.read_csv('sample_data/SUbi.xls')

# Load the second CSV file
df2 = pd.read_csv('sample_data/SID.xls')

# Merge the two dataframes on the common column 'SensorID' and 'MQ_KURZNAME'
merged_df = pd.merge(df2, df1, left_on='SensorID', right_on='MQ_KURZNAME')

# Drop duplicate rows based on SensorID
unique_merged_df = merged_df.drop_duplicates(subset='SensorID')

# Drop unnecessary columns from the final dataframe
unique_merged_df = unique_merged_df.drop(columns=['MQ_KURZNAME'])

unique_merged_df.to_csv('Senor_info_map.csv')

# Extract numeric IDs from the 'numeric_id' column of the DataFrame
sensor_list = unique_merged_df['Numeric_ID'].tolist()

# Example of intensity values, in reality we would have yhats
intensity_values = [random.random() for _ in range(len(sensor_list))]

# Create a DataFrame with 'Numeric_ID' and 'Intensity' columns
input_df = pd.DataFrame({'Numeric_ID': sensor_list, 'Intensity': intensity_values})

# Write the DataFrame to a CSV file
input_df.to_csv('Example_input.csv', index=False)


display(input_df.head())
display(unique_merged_df.head())
# Merge the previous DataFrame with the new DataFrame on 'SensorID'

merged_df = pd.merge(input_df, unique_merged_df, on='Numeric_ID')


# Rename columns
merged_df.rename(columns={'LÄNGE (WGS84)': 'lon', 'BREITE (WGS84)': 'lat', 'intensity': 'Intensity'}, inplace=True)


display(merged_df.head())

,Numeric_ID,Intensity
0,1,0.387914
1,2,0.368655
2,3,0.224942
3,4,0.540844
4,5,0.693184


,SensorID,Numeric_ID,LÄNGE (WGS84),BREITE (WGS84)
0,TE002,1,13.192747,52.433813
2,TE005,2,13.263106,52.435112
4,TE042,3,13.365083,52.559064
5,TE162,4,13.440941,52.524298
7,TE006,5,13.259881,52.433741


,Numeric_ID,Intensity,SensorID,lon,lat
0,1,0.387914,TE002,13.192747,52.433813
1,2,0.368655,TE005,13.263106,52.435112
2,3,0.224942,TE042,13.365083,52.559064
3,4,0.540844,TE162,13.440941,52.524298
4,5,0.693184,TE006,13.259881,52.433741


In [4]:
import numpy as np
import folium
from folium.plugins import HeatMap
from branca.colormap import LinearColormap


berlin_latitudes = merged_df['lat'].tolist()
berlin_longitudes = merged_df['lon'].tolist()
intensities = merged_df['Intensity'].tolist()

# Create a Folium map centered around Berlin
berlin_map = folium.Map(location=[52.52, 13.405], zoom_start=12)

# Add heatmap layer with intensity values
heatmap_data = list(zip(berlin_latitudes, berlin_longitudes, intensities))
HeatMap(heatmap_data).add_to(berlin_map)

# Define a colormap for intensity values
colormap = LinearColormap(['blue', 'green', 'yellow', 'orange', 'red'], vmin=min(intensities), vmax=max(intensities))
colormap.caption = 'Intensity'

# Add the colormap legend to the map
berlin_map.add_child(colormap)


# Save the map to an HTML file
berlin_map.save("heatmap_berlin_with_intensity_and_legend.html")
